# Check device

In [1]:
######################################################

# Check device
!nvidia-smi

Thu Mar  3 19:10:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Package Install

In [2]:
# ====================================================
# Install
# ====================================================

!pip install -q datasets==1.18.3
!pip install -q sentencepiece==0.1.96
!pip install -q transformers==4.16.2

     |████████████████████████████████| 311 kB 13.7 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 61.0 MB/s 
     |████████████████████████████████| 134 kB 86.2 MB/s 
     |████████████████████████████████| 212 kB 88.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 68.0 MB/s 
     |████████████████████████████████| 144 kB 70.5 MB/s 
     |████████████████████████████████| 1.2 MB 15.0 MB/s 
     |████████████████████████████████| 3.5 MB 14.1 MB/s 
     |████████████████████████████████| 596 kB 76.8 MB/s 
     |████████████████████████████████| 895 kB 70.9 MB/s 
     |████████████████████████████████| 6.5 MB 77.9 MB/s 


# Config

In [3]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=30
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=2
    fc_dropout=0.2
    max_len=450
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    # MLM setting
    mlm_probability=0.40 # 0.15
    max_seq_length=None
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [4]:
# ====================================================
# Define path
# ====================================================
import os

INPUT_DIR = '/content/drive/MyDrive/kaggle-nbme-score-clinical-patient-notes/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle-nbme-score-clinical-patient-notes/pretrained/{}/'.format(CFG.model.replace('/', '-'))
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Library

In [5]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForMaskedLM
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers.modeling_outputs import MaskedLMOutput
from transformers import DataCollatorForLanguageModeling
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


# Utils

In [6]:
# ====================================================
# Utils
# ===================================================
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

# Trainer

In [7]:
######################################################

# Trainer
def trainer(model, data_loader, optimizer, scheduler, CFG):
    model.train()

    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = []

    optimizer.zero_grad()

    for idx, batch in enumerate(data_loader):
        for k, v in batch.items():
            batch[k] = v.to(device, dtype=torch.long)

        with torch.cuda.amp.autocast(enabled=CFG.apex):
            outputs = model(**batch)

        loss = outputs.loss

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (idx + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            CFG.global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        
        losses.append(loss.detach().cpu().item())

        # if (CFG.global_step % CFG.save_step) == 0:
        #     LOGGER.info(
        #         "Epoch {} Step {}: Train Loss {:.4f}, elapsed {:.4f}s".format(
        #             CFG.epoch + 1, CFG.global_step, np.mean(losses), time.time() - start)
        #         )
        #     torch.save(
        #         model.state_dict(),
        #         OUTPUT_DIR + '{}-mlm-step-{}.bin'.format(
        #             CFG.model.replace('/', '-'),
        #             CFG.global_step))
        
        if idx % 500 == 0 or idx == (len(data_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss:.4f} '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, idx, len(data_loader), 
                          remain=timeSince(start, float(idx+1)/len(data_loader)),
                          loss=np.mean(losses),
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
    
    return np.mean(losses)

# Model

In [8]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "model" not in n],
         'lr': decoder_lr, 'weight_decay': 0.0}
    ]
    return optimizer_parameters

In [9]:
def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler=='linear':
        scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler=='cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
    return scheduler

In [10]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(
                cfg.model,
                output_hidden_states=False
                )
        else:
            self.config = torch.load(config_path)
        
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            self.lm_head = AutoModelForMaskedLM.from_pretrained(cfg.model, config=self.config).cls # [cls, lm_head]
        else:
            self.model = AutoModel(self.config)
            self.lm_head = AutoModelForMaskedLM(self.config).cls # [cls, lm_head]
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(
            self, 
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            #position_ids=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None):
        
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            #position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,)
        
        sequence_output = outputs[0]
        prediction_scores = self.lm_head(sequence_output)

        masked_lm_loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

        return MaskedLMOutput(loss=masked_lm_loss,
                              logits=prediction_scores,
                              hidden_states=outputs.hidden_states,
                              attentions=outputs.attentions)

# Main

In [11]:
# ====================================================
# data loading
# ====================================================
train = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))
features = pd.read_csv(os.path.join(INPUT_DIR, 'features.csv'))
patient_notes = pd.read_csv(os.path.join(INPUT_DIR, 'patient_notes.csv'))
display(patient_notes.head())
display(patient_notes['pn_history'].nunique(), len(patient_notes))

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


42146

42146

In [12]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
CFG.tokenizer = tokenizer

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
mlm_data = patient_notes[['pn_history']].reset_index(drop=True)
mlm_data = mlm_data.rename(columns={'pn_history': 'text'})
csv_name = f'mlm_data.csv'
mlm_data.to_csv(OUTPUT_DIR+csv_name, index=False)
print(f"Saved mlm data: {csv_name}")
print(f"mlm data: {mlm_data.shape}")

Saved mlm data: mlm_data.csv
mlm data: (42146, 1)


In [14]:
#####################################################

# Training support

def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True)

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // CFG.max_len) * CFG.max_len
    result = {
        k: [t[i : i + CFG.max_len] for i in range(0, total_length, CFG.max_len)]
        for k, t in concatenated_examples.items()
    }
    return result

In [15]:
seed_everything(CFG.seed)

CFG.train_file = f"mlm_data.csv"
data_files = {'train': OUTPUT_DIR+CFG.train_file}
raw_datasets = load_dataset('csv', data_files=data_files)

Using custom data configuration default-2954b82d524aa7c6


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2954b82d524aa7c6/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
if CFG.max_seq_length is None:
    max_seq_length = tokenizer.model_max_length
else:
    if CFG.max_seq_length > tokenizer.model_max_length:
        max_seq_length = min(CFG.max_seq_length, tokenizer.model_max_length)

In [17]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"],
    load_from_cache_file=not True,
    )
LOGGER.info(f"tokenized_datasets: {tokenized_datasets}")

Setting TOKENIZERS_PARALLELISM=false for forked processes.
tokenized_datasets: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 42146
    })
})


In [18]:
tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
    load_from_cache_file=not True,
    )
train_dataset = tokenized_datasets["train"]
LOGGER.info(f"train_dataset: {train_dataset}")

Setting TOKENIZERS_PARALLELISM=false for forked processes.
train_dataset: Dataset({
    features: ['input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 17763
})


In [19]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=CFG.mlm_probability
    )
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=CFG.batch_size
    )

In [20]:
model = CustomModel(CFG, config_path=None, pretrained=True)
model.to(device)
optimizer_parameters = get_optimizer_params(
    model,
    encoder_lr=CFG.encoder_lr, 
    decoder_lr=CFG.decoder_lr,
    weight_decay=CFG.weight_decay)
optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

Downloading:   0%|          | 0.00/833M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the 

In [21]:
num_train_steps = int(len(mlm_data) / CFG.batch_size * CFG.epochs)
scheduler = get_scheduler(CFG, optimizer, num_train_steps)

In [22]:
print(num_train_steps)

632190


In [ ]:
CFG.global_step = 0
CFG.save_step = 25000

for epoch in range(CFG.epochs):
    CFG.epoch = epoch
    start = time.time()
        
    train_loss = trainer(model, train_loader, optimizer, scheduler, CFG)

    LOGGER.info(
        "Epoch {}: Train Loss {:.4f}, elapsed {:.4f}s".format(
            epoch + 1, train_loss, time.time() - start)
        )
    torch.save(
        model.state_dict(),
        OUTPUT_DIR + '{}-mlm-epoch-{}.bin'.format(
            CFG.model.replace('/', '-'),
            CFG.epoch + 1))

Epoch: [1][0/8882] Elapsed 0m 1s (remain 167m 25s) Loss: 12.2465 Grad: inf  LR: 0.00002000  
Epoch: [1][500/8882] Elapsed 2m 26s (remain 40m 58s) Loss: 5.5722 Grad: 56626.3945  LR: 0.00002000  
Epoch: [1][1000/8882] Elapsed 4m 53s (remain 38m 27s) Loss: 4.6518 Grad: 69817.8125  LR: 0.00002000  
Epoch: [1][1500/8882] Elapsed 7m 19s (remain 35m 58s) Loss: 4.1503 Grad: 54568.5820  LR: 0.00002000  
Epoch: [1][2000/8882] Elapsed 9m 45s (remain 33m 31s) Loss: 3.8168 Grad: 53982.0547  LR: 0.00002000  
Epoch: [1][2500/8882] Elapsed 12m 11s (remain 31m 5s) Loss: 3.5697 Grad: 110480.6406  LR: 0.00002000  
Epoch: [1][3000/8882] Elapsed 14m 37s (remain 28m 38s) Loss: 3.3927 Grad: 114175.9922  LR: 0.00002000  
Epoch: [1][3500/8882] Elapsed 17m 3s (remain 26m 12s) Loss: 3.2527 Grad: 111778.2344  LR: 0.00002000  
Epoch: [1][4000/8882] Elapsed 19m 30s (remain 23m 47s) Loss: 3.1380 Grad: 106983.9531  LR: 0.00002000  
Epoch: [1][4500/8882] Elapsed 21m 56s (remain 21m 21s) Loss: 3.0386 Grad: 240411.4688 

Epoch 1: Train Loss 2.5651, elapsed 2588.3083s


Epoch: [1][8881/8882] Elapsed 43m 8s (remain 0m 0s) Loss: 2.5651 Grad: 548954.3125  LR: 0.00001999  
Epoch: [2][0/8882] Elapsed 0m 0s (remain 50m 32s) Loss: 1.9792 Grad: 408525.0000  LR: 0.00001999  
Epoch: [2][500/8882] Elapsed 2m 25s (remain 40m 32s) Loss: 1.9540 Grad: 431627.7812  LR: 0.00001999  
Epoch: [2][1000/8882] Elapsed 4m 49s (remain 38m 1s) Loss: 1.9418 Grad: 440722.0000  LR: 0.00001999  
Epoch: [2][1500/8882] Elapsed 7m 14s (remain 35m 34s) Loss: 1.9275 Grad: 416848.3750  LR: 0.00001999  
Epoch: [2][2000/8882] Elapsed 9m 38s (remain 33m 9s) Loss: 1.9186 Grad: 862945.9375  LR: 0.00001999  
Epoch: [2][2500/8882] Elapsed 12m 2s (remain 30m 44s) Loss: 1.9113 Grad: 396378.0000  LR: 0.00001998  
Epoch: [2][3000/8882] Elapsed 14m 26s (remain 28m 18s) Loss: 1.9063 Grad: 440765.7500  LR: 0.00001998  
Epoch: [2][3500/8882] Elapsed 16m 50s (remain 25m 53s) Loss: 1.8990 Grad: 472494.3125  LR: 0.00001998  
Epoch: [2][4000/8882] Elapsed 19m 15s (remain 23m 29s) Loss: 1.8923 Grad: 249725

Epoch 2: Train Loss 1.8150, elapsed 2566.3125s


Epoch: [2][8881/8882] Elapsed 42m 46s (remain 0m 0s) Loss: 1.8150 Grad: 571499.0000  LR: 0.00001996  
Epoch: [3][0/8882] Elapsed 0m 0s (remain 52m 31s) Loss: 1.5482 Grad: 554923.5625  LR: 0.00001996  
Epoch: [3][500/8882] Elapsed 2m 26s (remain 40m 46s) Loss: 1.6868 Grad: 363727.4375  LR: 0.00001996  
Epoch: [3][1000/8882] Elapsed 4m 51s (remain 38m 13s) Loss: 1.6871 Grad: 475128.2500  LR: 0.00001996  
Epoch: [3][1500/8882] Elapsed 7m 16s (remain 35m 44s) Loss: 1.6788 Grad: 414488.5000  LR: 0.00001995  
Epoch: [3][2000/8882] Elapsed 9m 40s (remain 33m 17s) Loss: 1.6758 Grad: 791860.7500  LR: 0.00001995  
Epoch: [3][2500/8882] Elapsed 12m 5s (remain 30m 52s) Loss: 1.6666 Grad: 362081.9062  LR: 0.00001995  
Epoch: [3][3000/8882] Elapsed 14m 30s (remain 28m 26s) Loss: 1.6616 Grad: 397199.5000  LR: 0.00001995  
Epoch: [3][3500/8882] Elapsed 16m 55s (remain 26m 1s) Loss: 1.6580 Grad: 360379.3750  LR: 0.00001994  
Epoch: [3][4000/8882] Elapsed 19m 20s (remain 23m 36s) Loss: 1.6566 Grad: 2047

...

Epoch: [22][8881/8882] Elapsed 42m 52s (remain 0m 0s) Loss: 1.1251 Grad: 85824.1406  LR: 0.00001564  
Epoch: [23][0/8882] Elapsed 0m 0s (remain 52m 53s) Loss: 0.8431 Grad: 251446.9844  LR: 0.00001564  
Epoch: [23][500/8882] Elapsed 2m 25s (remain 40m 39s) Loss: 1.1023 Grad: 290779.3125  LR: 0.00001562  
Epoch: [23][1000/8882] Elapsed 4m 50s (remain 38m 3s) Loss: 1.0940 Grad: 301840.3438  LR: 0.00001560  
Epoch: [23][1500/8882] Elapsed 7m 14s (remain 35m 37s) Loss: 1.0919 Grad: 232985.8750  LR: 0.00001558  
Epoch: [23][2000/8882] Elapsed 9m 38s (remain 33m 11s) Loss: 1.0905 Grad: 485465.1875  LR: 0.00001556  
Epoch: [23][2500/8882] Elapsed 12m 4s (remain 30m 48s) Loss: 1.0906 Grad: 242799.2969  LR: 0.00001554  
Epoch: [23][3000/8882] Elapsed 14m 29s (remain 28m 24s) Loss: 1.0879 Grad: 272803.7500  LR: 0.00001552  
Epoch: [23][3500/8882] Elapsed 16m 54s (remain 25m 59s) Loss: 1.0894 Grad: 139269.7812  LR: 0.00001550  
Epoch: [23][4000/8882] Elapsed 19m 18s (remain 23m 33s) Loss: 1.0909 G

Epoch 23: Train Loss 1.1000, elapsed 2579.9142s


Epoch: [23][8881/8882] Elapsed 42m 59s (remain 0m 0s) Loss: 1.1000 Grad: 175999.1094  LR: 0.00001527  
Epoch: [24][0/8882] Elapsed 0m 0s (remain 51m 43s) Loss: 1.1286 Grad: 288720.8125  LR: 0.00001527  
Epoch: [24][500/8882] Elapsed 2m 27s (remain 41m 11s) Loss: 1.0784 Grad: 270819.8125  LR: 0.00001525  
Epoch: [24][1000/8882] Elapsed 4m 53s (remain 38m 33s) Loss: 1.0837 Grad: 300673.1562  LR: 0.00001523  
Epoch: [24][1500/8882] Elapsed 7m 19s (remain 36m 0s) Loss: 1.0788 Grad: 269994.3125  LR: 0.00001521  
Epoch: [24][2000/8882] Elapsed 9m 45s (remain 33m 33s) Loss: 1.0773 Grad: 647841.3750  LR: 0.00001519  
Epoch: [24][2500/8882] Elapsed 12m 11s (remain 31m 6s) Loss: 1.0755 Grad: 631090.8125  LR: 0.00001517  
Epoch: [24][3000/8882] Elapsed 14m 37s (remain 28m 40s) Loss: 1.0741 Grad: 558113.7500  LR: 0.00001515  
Epoch: [24][3500/8882] Elapsed 17m 3s (remain 26m 13s) Loss: 1.0728 Grad: 600946.4375  LR: 0.00001513  
Epoch: [24][4000/8882] Elapsed 19m 29s (remain 23m 46s) Loss: 1.0724 G

Epoch 24: Train Loss 1.0734, elapsed 2589.5395s


Epoch: [24][8881/8882] Elapsed 43m 9s (remain 0m 0s) Loss: 1.0734 Grad: 630925.3125  LR: 0.00001489  
Epoch: [25][0/8882] Elapsed 0m 0s (remain 54m 15s) Loss: 1.1779 Grad: 284410.9688  LR: 0.00001489  
Epoch: [25][500/8882] Elapsed 2m 26s (remain 40m 47s) Loss: 1.0763 Grad: 317287.3438  LR: 0.00001487  
Epoch: [25][1000/8882] Elapsed 4m 51s (remain 38m 12s) Loss: 1.0736 Grad: 268723.8750  LR: 0.00001485  
Epoch: [25][1500/8882] Elapsed 7m 15s (remain 35m 41s) Loss: 1.0735 Grad: 296154.9375  LR: 0.00001483  
Epoch: [25][2000/8882] Elapsed 9m 40s (remain 33m 14s) Loss: 1.0701 Grad: 547060.0625  LR: 0.00001481  
Epoch: [25][2500/8882] Elapsed 12m 4s (remain 30m 48s) Loss: 1.0703 Grad: 618426.1250  LR: 0.00001479  
Epoch: [25][3000/8882] Elapsed 14m 28s (remain 28m 22s) Loss: 1.0695 Grad: 820062.1875  LR: 0.00001476  
Epoch: [25][3500/8882] Elapsed 16m 53s (remain 25m 57s) Loss: 1.0660 Grad: 514933.1875  LR: 0.00001474  
Epoch: [25][4000/8882] Elapsed 19m 17s (remain 23m 32s) Loss: 1.0675 